## [Przykładowy notatnik został zaczerpnięty z dokumentacji Sparka](https://spark.apache.org/docs/latest/ml-classification-regression.html#naive-bayes)

### Importowanie potrzebnych bibliotek

In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

### Inicjowanie sesji

In [2]:
findspark.init()
spark = SparkSession.builder.getOrCreate()

### Ładowanie zbioru danych



_[Zbiór danych został zaczerpnięty z oficjalnego repozytorium Sparka](https://github.com/apache/spark/blob/master/data/mllib/sample_libsvm_data.txt)_

In [3]:
data = spark.read.format("libsvm").load("/home/jovyan/work/sample_libsvm_data.txt")

### Podział danych na zestaw treningowy i zestaw testowy

Zbiór danych zostanie losowo podzielony na dwa zbiory. Zbiór treningowy będzie zawierał 70% danych z początkowego zbioru. Zostanie on wykorzystany do zbudowania modelu klasyfikującego. Zbiór testowy będzie zawierał 30% danych z początkowego zbioru. Zostanie on wykorzystany do ewaluacji zbudowanego modelu. Po podziale wypisywana jest ilość instancji znajdujących się w każdym ze zbiorów.

In [ ]:
splits = data.randomSplit([0.6, 0.4], 1234)
train = splits[0]
test = splits[1]

print(f"Ilość instancji w zbiorze treningowym: {train.count()}")
print(f"Ilość instancji w zbiorze testowym: {test.count()}")

Ilość instancji w zbiorze treningowym: 58
Ilość instancji w zbiorze testowym: 42


### Tworzenie i trenowanie wieloklasowego klasyfikatora przy użyciu algorytmu Naive Bayes

Klasyfikator ten spróbuje przewidzieć jakiej kategorii jest dana instancja, na podstawie jej cech charakterystycznych. __Należy pamiętać, że ten algorytm _przyjmuje_, że podane mu cechu są od siebie niezależne i nieujemne__. Przy tworzeniu modelu zostanie wykorzystany potok z biblioteki MLlib środowiska Spark.

In [5]:
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")
model = nb.fit(train)

### Uruchomienie zbudowanego modelu na danych testowych

Na podstawie zdobytej wiedzy, model przewiduje kategorie instancji ze zbioru testowego.

In [6]:
predictions = model.transform(test)

### Prezentacja przewidywań

W kolumnie _label_ znajduje się faktyczny numer kategorii. W kolumnie _features_ znajduje się wektor cech charakterystycznych. W kolumnie _rawPrediction_ znajduje się wektor zawierający _surowe_ wartości przewidywań. W kolumnie _probability_ znajduje się wektor z prawdopodobieństwami, z jakimi dana instancja należy do każdej z kategorii. W kolumnie _prediction_ znajduje się przewidywany numer kategorii (nadany instancji przez klasyfikator).

In [10]:
predictions.show()

+-----+--------------------+--------------------+-----------+----------+
|label|            features|       rawPrediction|probability|prediction|
+-----+--------------------+--------------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|[-172664.79564650...|  [1.0,0.0]|       0.0|
|  0.0|(692,[98,99,100,1...|[-176279.15054306...|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,124...|[-189600.55409526...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-274673.88337431...|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[-183393.03869049...|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|[-256992.48807619...|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[-210411.53649773...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-170627.63616681...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-212157.96750469...|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|[-183253.80108550...|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|[-246528.93739632...|  

### Ewaluacja modelu

Do ewaluacji badanego modelu wykorzystano ewaluator typu _MulticlassClassificationEvaluator_ z biblioteki MLlib środowiska Spark. Posłużył on do obliczenia i wyświetlenia dokładności zbudowanego modelu.

In [8]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Dokładność modelu wynosi około {round(accuracy * 100, 2)}%")

Dokładność modelu wynosi około 100.0%


### Zatrzymywanie sesji

In [11]:
spark.stop()